In [1]:
import numpy as np
import pandas as pd
from PIL import Image
from tqdm import tqdm
import glob
import cv2
import torch


from torch import nn
from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler
from torchvision import transforms
import torchvision.models as models
from torch.optim import lr_scheduler

from efficientnet_pytorch import EfficientNet
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from cutmix.cutmix import CutMix
from cutmix.utils import CutMixCrossEntropyLoss

import warnings

warnings.filterwarnings("ignore")

C:\Users\user\.conda\envs\new_dacon\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class CustomDataset(Dataset):
    def __init__(self, files, labels=None, mode='train', transform=None):
        self.mode = mode
        self.files = files # img_path_list
        if mode == 'train':
            self.labels = labels # label_list
        self.transform = transform

    def __len__(self):
        return len(self.files)

    def __getitem__(self, i):
        if self.mode == 'train':
            img = Image.open(self.files[i])

            if self.transform:
                img = self.transform(img)

            return {
                'img': torch.tensor(img, dtype=torch.float32).clone().detach(),
                'label': torch.tensor(self.labels[i], dtype=torch.long)
            }
        else:
            img = Image.open(self.files[i])

            if self.transform:
                img = self.transform(img)

            return {
                'img': torch.tensor(img, dtype=torch.float32).clone().detach(),
            }

In [3]:
mytransform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.RandomPerspective(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

myvaltransform =transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [4]:
class CNN_Model(nn.Module):
    def __init__(self, class_n, rate=0.2):
        super(CNN_Model, self).__init__()
#         self.model = EfficientNet.from_pretrained('efficientnet-b7')
        self.model = models.efficientnet_v2_l(pretrained=True)
        self.dropout = nn.Dropout(rate)
        self.output_layer = nn.Linear(in_features=1000, out_features=class_n, bias=True)

    def forward(self, inputs):
        output = self.output_layer(self.dropout(self.model(inputs)))
        return output

In [5]:
def train_step(model, batch_item, epoch, batch, training,class_weight=None):
    img = batch_item['img'].to(device)
    label = batch_item['label'].to(device)
    if training is True:
        model.train()
        optimizer.zero_grad()
        with torch.cuda.amp.autocast():
            output = model(img)
            loss = criterion(output, label,class_weight=class_weight)
        loss.backward()
        optimizer.step()
        scheduler.step()

        return loss
    else:
        model.eval()
        with torch.no_grad():
            output = model(img)
            loss = criterion(output, label,class_weight=class_weight)
            return loss
            
def predict(models,dataset):
    for fold,model in enumerate(models):
        model.eval()
    tqdm_dataset = tqdm(enumerate(dataset))
    training = False
    results = []
    for batch, batch_item in tqdm_dataset:
        img = batch_item['img'].to(device)
        for fold,model in enumerate(models):
            with torch.no_grad():
                if fold ==0:
                    output = model(img)
                else:
                    output = output+model(img)
        output = 0.2*output
        output = torch.tensor(torch.argmax(output, axis=-1), dtype=torch.int32).cpu().numpy()
        results.extend(output)
    return results

In [6]:
all_img_list = glob.glob('./train/*/*')

df = pd.DataFrame(columns=['img_path', 'label'])
df['img_path'] = all_img_list
# print(df['img_path'])
# print(str(df['img_path'][0]).split('\\'))
df['label'] = df['img_path'].apply(lambda x : str(x).split('\\')[1])

In [7]:
# Load Data


# Hyperparameter Setting
device = torch.device("cuda")

CFG = {
    'IMG_SIZE':224,
    'EPOCHS':100,
    'LEARNING_RATE':2e-4,
    'BATCH_SIZE':32,
    'SEED':100,
    'FOLD' : 5,
}


# batch_size = 32
# learning_rate = 2e-4
# epochs = 300
# folds = 5
# random_seed=100
# class_n = len(train_total['disease_code'].unique())

save_path = 'models/model.'

train, val, _, _ = train_test_split(df, df['label'], test_size=0.3, stratify=df['label'], random_state=CFG['SEED'])

le = preprocessing.LabelEncoder()
train['label'] = le.fit_transform(train['label'])
val['label'] = le.transform(val['label'])
class_n = len(le.classes_)
# Cross Validation
kfold = StratifiedKFold(n_splits=CFG['FOLD'],shuffle=True,random_state=CFG['SEED'])
# Loss Weights
class_weight = torch.FloatTensor(1/train['label'].value_counts())

torch.manual_seed(CFG['SEED'])

In [8]:
train_dataset = CustomDataset(files=train['img_path'].str.split('/').str[-1].values, labels=train['label'].values,
                              transform =mytransform)

train_dataset = CutMix(train_dataset,num_class=class_n,beta=1.0,prob=0.5,num_mix=2)
valid_dataset = CustomDataset(files=train['img_path'].str.split('/').str[-1].values, labels=train['label'].values,
                              transform=myvaltransform)

test = pd.read_csv('./test.csv')

test_dataset = CustomDataset(files=test['img_path'].values, labels=None, mode ='test', transform = myvaltransform)
test_dataloader = DataLoader(test_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

In [9]:
k_loss_plot, k_val_loss_plot = [], []    
for fold,(train_idx,valid_idx) in enumerate(kfold.split(train_dataset,train['label'])):
    train_subsampler = SubsetRandomSampler(train_idx)
    valid_subsampler = SubsetRandomSampler(valid_idx)

    train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=CFG['BATCH_SIZE'], num_workers=0, sampler=train_subsampler)
    val_dataloader = torch.utils.data.DataLoader(valid_dataset, batch_size=CFG['BATCH_SIZE'], num_workers=0, sampler=valid_subsampler)

    model = CNN_Model(class_n).to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=CFG["LEARNING_RATE"])
    criterion = CutMixCrossEntropyLoss(True)
    scheduler = lr_scheduler.CosineAnnealingLR(optimizer, T_max=10)

    loss_plot, val_loss_plot = [], []
    # Training
    for epoch in range(CFG["EPOCHS"]):
        total_loss, total_val_loss = 0, 0
        tqdm_dataset = tqdm(enumerate(train_dataloader))
        training = True
        for batch, batch_item in tqdm_dataset:
            batch_loss = train_step(model,batch_item, epoch, batch, training,class_weight=class_weight)
            total_loss += batch_loss.item()

            tqdm_dataset.set_postfix({
                'Epoch': epoch + 1,
                'Loss': '{:06f}'.format(batch_loss.item()),
                'Total Loss': '{:06f}'.format(total_loss / (batch + 1))
            })
        loss_plot.append((total_loss / (batch + 1)))

        tqdm_dataset = tqdm(enumerate(val_dataloader))
        training = False
        for batch, batch_item in tqdm_dataset:
            batch_loss = train_step(model,batch_item, epoch, batch, training)
            total_val_loss += batch_loss.item()

            tqdm_dataset.set_postfix({
                'Epoch': epoch + 1,
                'Val Loss': '{:06f}'.format(batch_loss.item()),
                'Total Val Loss': '{:06f}'.format(total_val_loss / (batch + 1))
            })
        val_loss_plot.append((total_val_loss / (batch + 1)))

        if np.min(val_loss_plot) == val_loss_plot[-1]:
            torch.save(model.state_dict(), save_path+str(fold)+".pt")

    k_loss_plot.append(min(loss_plot))
    k_val_loss_plot.append(min(val_loss_plot))

print("Train Loss: ",np.mean(k_loss_plot),", Valid Loss: ",np.mean(k_val_loss_plot))

61it [01:50,  1.82s/it, Epoch=1, Loss=1.074308, Total Loss=1.664600]
16it [00:06,  2.33it/s, Epoch=1, Val Loss=1.102427, Total Val Loss=1.486103]
61it [01:47,  1.76s/it, Epoch=2, Loss=1.598105, Total Loss=1.352990]
16it [00:06,  2.35it/s, Epoch=2, Val Loss=1.970047, Total Val Loss=1.110258]
61it [01:47,  1.76s/it, Epoch=3, Loss=0.981109, Total Loss=1.225883]
16it [00:06,  2.35it/s, Epoch=3, Val Loss=0.746686, Total Val Loss=0.870076]
61it [01:47,  1.76s/it, Epoch=4, Loss=1.347518, Total Loss=1.112329]
16it [00:06,  2.37it/s, Epoch=4, Val Loss=1.304238, Total Val Loss=0.846294]
61it [01:44,  1.72s/it, Epoch=5, Loss=1.217604, Total Loss=1.064408]
16it [00:06,  2.29it/s, Epoch=5, Val Loss=1.481728, Total Val Loss=0.891851]
61it [01:47,  1.75s/it, Epoch=6, Loss=1.499221, Total Loss=1.043358]
16it [00:06,  2.37it/s, Epoch=6, Val Loss=0.588534, Total Val Loss=0.736043]
61it [01:46,  1.75s/it, Epoch=7, Loss=0.599746, Total Loss=0.954147]
16it [00:06,  2.31it/s, Epoch=7, Val Loss=2.059761, Tot

16it [00:06,  2.48it/s, Epoch=56, Val Loss=0.006640, Total Val Loss=0.220897]
61it [01:41,  1.67s/it, Epoch=57, Loss=0.748347, Total Loss=0.556048]
16it [00:06,  2.44it/s, Epoch=57, Val Loss=0.045458, Total Val Loss=0.203054]
61it [01:40,  1.65s/it, Epoch=58, Loss=0.551010, Total Loss=0.562224]
16it [00:06,  2.52it/s, Epoch=58, Val Loss=0.251012, Total Val Loss=0.232087]
61it [01:40,  1.65s/it, Epoch=59, Loss=0.909080, Total Loss=0.557741]
16it [00:06,  2.48it/s, Epoch=59, Val Loss=0.039619, Total Val Loss=0.178230]
61it [01:40,  1.65s/it, Epoch=60, Loss=0.591404, Total Loss=0.551207]
16it [00:06,  2.36it/s, Epoch=60, Val Loss=0.021957, Total Val Loss=0.198788]
61it [01:43,  1.69s/it, Epoch=61, Loss=0.724513, Total Loss=0.559885]
16it [00:06,  2.47it/s, Epoch=61, Val Loss=0.013059, Total Val Loss=0.204089]
61it [01:41,  1.66s/it, Epoch=62, Loss=0.682095, Total Loss=0.539844]
16it [00:06,  2.53it/s, Epoch=62, Val Loss=0.029255, Total Val Loss=0.195412]
61it [01:41,  1.67s/it, Epoch=63, 

16it [00:06,  2.48it/s, Epoch=11, Val Loss=0.221596, Total Val Loss=0.602055]
61it [01:41,  1.66s/it, Epoch=12, Loss=1.200800, Total Loss=0.838320]
16it [00:06,  2.44it/s, Epoch=12, Val Loss=0.928870, Total Val Loss=0.585866]
61it [01:41,  1.67s/it, Epoch=13, Loss=1.124433, Total Loss=0.835819]
16it [00:06,  2.48it/s, Epoch=13, Val Loss=0.102732, Total Val Loss=0.597521]
61it [01:40,  1.64s/it, Epoch=14, Loss=0.662086, Total Loss=0.766115]
16it [00:06,  2.47it/s, Epoch=14, Val Loss=1.209069, Total Val Loss=0.636855]
61it [01:42,  1.68s/it, Epoch=15, Loss=0.546676, Total Loss=0.806118]
16it [00:06,  2.48it/s, Epoch=15, Val Loss=0.082464, Total Val Loss=0.488225]
61it [01:41,  1.67s/it, Epoch=16, Loss=0.506164, Total Loss=0.780587]
16it [00:06,  2.52it/s, Epoch=16, Val Loss=0.003862, Total Val Loss=0.468546]
61it [01:39,  1.63s/it, Epoch=17, Loss=1.017441, Total Loss=0.766016]
16it [00:06,  2.44it/s, Epoch=17, Val Loss=0.560875, Total Val Loss=0.482492]
61it [01:39,  1.63s/it, Epoch=18, 

16it [00:06,  2.53it/s, Epoch=66, Val Loss=0.244614, Total Val Loss=0.238538]
61it [01:39,  1.64s/it, Epoch=67, Loss=0.573856, Total Loss=0.502940]
16it [00:06,  2.52it/s, Epoch=67, Val Loss=0.013006, Total Val Loss=0.231577]
61it [01:40,  1.64s/it, Epoch=68, Loss=0.427791, Total Loss=0.492468]
16it [00:06,  2.51it/s, Epoch=68, Val Loss=0.061728, Total Val Loss=0.206497]
61it [01:38,  1.62s/it, Epoch=69, Loss=0.771869, Total Loss=0.520389]
16it [00:06,  2.48it/s, Epoch=69, Val Loss=0.039282, Total Val Loss=0.253550]
61it [01:39,  1.63s/it, Epoch=70, Loss=0.569906, Total Loss=0.544524]
16it [00:06,  2.51it/s, Epoch=70, Val Loss=0.073538, Total Val Loss=0.288461]
61it [01:41,  1.66s/it, Epoch=71, Loss=0.244407, Total Loss=0.496880]
16it [00:06,  2.54it/s, Epoch=71, Val Loss=0.003780, Total Val Loss=0.198626]
61it [01:53,  1.87s/it, Epoch=72, Loss=1.019991, Total Loss=0.545538]
16it [00:09,  1.65it/s, Epoch=72, Val Loss=0.004883, Total Val Loss=0.175751]
61it [01:49,  1.80s/it, Epoch=73, 

16it [00:06,  2.47it/s, Epoch=21, Val Loss=0.875977, Total Val Loss=0.459318]
61it [01:40,  1.65s/it, Epoch=22, Loss=0.924963, Total Loss=0.683055]
16it [00:06,  2.48it/s, Epoch=22, Val Loss=0.043598, Total Val Loss=0.449862]
61it [01:39,  1.64s/it, Epoch=23, Loss=0.728153, Total Loss=0.701442]
16it [00:06,  2.48it/s, Epoch=23, Val Loss=0.488954, Total Val Loss=0.459553]
61it [01:40,  1.65s/it, Epoch=24, Loss=0.443954, Total Loss=0.670893]
16it [00:06,  2.48it/s, Epoch=24, Val Loss=0.006605, Total Val Loss=0.374124]
61it [01:40,  1.65s/it, Epoch=25, Loss=0.602253, Total Loss=0.675351]
16it [00:06,  2.49it/s, Epoch=25, Val Loss=0.970601, Total Val Loss=0.401176]
61it [01:41,  1.67s/it, Epoch=26, Loss=0.414910, Total Loss=0.708989]
16it [00:06,  2.47it/s, Epoch=26, Val Loss=0.298079, Total Val Loss=0.340831]
61it [01:39,  1.64s/it, Epoch=27, Loss=1.115321, Total Loss=0.627737]
16it [00:06,  2.52it/s, Epoch=27, Val Loss=1.117439, Total Val Loss=0.450062]
61it [01:41,  1.67s/it, Epoch=28, 

...

16it [00:06,  2.50it/s, Epoch=31, Val Loss=0.046787, Total Val Loss=0.337373]
61it [01:40,  1.65s/it, Epoch=32, Loss=0.549896, Total Loss=0.623435]
16it [00:06,  2.52it/s, Epoch=32, Val Loss=0.304336, Total Val Loss=0.308914]
61it [01:41,  1.66s/it, Epoch=33, Loss=0.764406, Total Loss=0.640710]
16it [00:06,  2.47it/s, Epoch=33, Val Loss=0.105679, Total Val Loss=0.315740]
61it [01:42,  1.69s/it, Epoch=34, Loss=0.559615, Total Loss=0.639035]
16it [00:06,  2.52it/s, Epoch=34, Val Loss=0.144128, Total Val Loss=0.306132]
61it [01:41,  1.66s/it, Epoch=35, Loss=0.561039, Total Loss=0.609334]
16it [00:06,  2.50it/s, Epoch=35, Val Loss=0.006537, Total Val Loss=0.308786]
61it [01:41,  1.66s/it, Epoch=36, Loss=1.301002, Total Loss=0.626913]
16it [00:06,  2.49it/s, Epoch=36, Val Loss=3.126354, Total Val Loss=0.470580]
61it [01:39,  1.64s/it, Epoch=37, Loss=0.548888, Total Loss=0.616431]
16it [00:06,  2.51it/s, Epoch=37, Val Loss=0.007215, Total Val Loss=0.284215]
61it [01:41,  1.66s/it, Epoch=38, 

16it [00:06,  2.53it/s, Epoch=86, Val Loss=1.746149, Total Val Loss=0.201267]
61it [01:40,  1.64s/it, Epoch=87, Loss=0.612543, Total Loss=0.477102]
16it [00:06,  2.55it/s, Epoch=87, Val Loss=0.104526, Total Val Loss=0.083440]
61it [01:39,  1.63s/it, Epoch=88, Loss=0.502809, Total Loss=0.488727]
16it [00:06,  2.54it/s, Epoch=88, Val Loss=0.043461, Total Val Loss=0.138078]
61it [01:40,  1.65s/it, Epoch=89, Loss=0.619561, Total Loss=0.505256]
16it [00:06,  2.53it/s, Epoch=89, Val Loss=0.088561, Total Val Loss=0.123589]
61it [01:41,  1.66s/it, Epoch=90, Loss=0.501026, Total Loss=0.517647]
16it [00:06,  2.52it/s, Epoch=90, Val Loss=0.027954, Total Val Loss=0.142189]
61it [01:39,  1.64s/it, Epoch=91, Loss=0.257571, Total Loss=0.490227]
16it [00:06,  2.50it/s, Epoch=91, Val Loss=0.019232, Total Val Loss=0.107762]
61it [01:40,  1.65s/it, Epoch=92, Loss=0.524258, Total Loss=0.517595]
16it [00:06,  2.54it/s, Epoch=92, Val Loss=0.068835, Total Val Loss=0.152332]
61it [01:38,  1.62s/it, Epoch=93, 

16it [00:06,  2.53it/s, Epoch=41, Val Loss=0.031705, Total Val Loss=0.296013]
61it [01:41,  1.66s/it, Epoch=42, Loss=0.636716, Total Loss=0.614329]
16it [00:06,  2.52it/s, Epoch=42, Val Loss=0.009110, Total Val Loss=0.360952]
61it [01:41,  1.66s/it, Epoch=43, Loss=0.468201, Total Loss=0.608841]
16it [00:06,  2.50it/s, Epoch=43, Val Loss=0.262800, Total Val Loss=0.348889]
61it [01:40,  1.65s/it, Epoch=44, Loss=0.382105, Total Loss=0.599302]
16it [00:06,  2.52it/s, Epoch=44, Val Loss=0.277164, Total Val Loss=0.307933]
61it [01:41,  1.67s/it, Epoch=45, Loss=0.563958, Total Loss=0.557864]
16it [00:06,  2.50it/s, Epoch=45, Val Loss=0.003470, Total Val Loss=0.329984]
61it [01:41,  1.66s/it, Epoch=46, Loss=0.574830, Total Loss=0.572764]
16it [00:06,  2.50it/s, Epoch=46, Val Loss=0.037818, Total Val Loss=0.278134]
61it [01:41,  1.67s/it, Epoch=47, Loss=0.828343, Total Loss=0.638120]
16it [00:06,  2.50it/s, Epoch=47, Val Loss=0.017447, Total Val Loss=0.275937]
61it [01:40,  1.65s/it, Epoch=48, 

16it [00:06,  2.43it/s, Epoch=96, Val Loss=0.010551, Total Val Loss=0.226510]
61it [01:39,  1.62s/it, Epoch=97, Loss=0.478457, Total Loss=0.465308]
16it [00:06,  2.50it/s, Epoch=97, Val Loss=0.000428, Total Val Loss=0.158730]
61it [01:40,  1.65s/it, Epoch=98, Loss=0.793728, Total Loss=0.485967]
16it [00:06,  2.51it/s, Epoch=98, Val Loss=0.002700, Total Val Loss=0.177586]
61it [01:40,  1.65s/it, Epoch=99, Loss=0.421354, Total Loss=0.525138]
16it [00:06,  2.47it/s, Epoch=99, Val Loss=0.004188, Total Val Loss=0.156031]
61it [01:40,  1.65s/it, Epoch=100, Loss=0.383761, Total Loss=0.475917]
16it [00:06,  2.52it/s, Epoch=100, Val Loss=0.036493, Total Val Loss=0.132424]


Train Loss:  0.46471782639378406 , Valid Loss:  0.10263231496792287


In [10]:
models_=[]
for i in range(5):
    model = CNN_Model(class_n).to(device)
    model.load_state_dict(torch.load(save_path+str(i)+".pt"))
    models_.append(model)
preds = predict(models_,test_dataloader)

25it [00:19,  1.31it/s]


In [11]:
print("[*] Submission")
submit = pd.read_csv('./sample_submission.csv')
submit['label'] = preds
submit.head()

[*] Submission


,id,label
0,TEST_000,18
1,TEST_001,10
2,TEST_002,18
3,TEST_003,18
4,TEST_004,10


In [12]:
submit.loc[submit['label'] == 0, 'label'] = '가구수정'
submit.loc[submit['label'] == 1, 'label'] = '걸레받이수정'
submit.loc[submit['label'] == 2, 'label'] = '곰팡이'
submit.loc[submit['label'] == 3, 'label'] = '꼬임'
submit.loc[submit['label'] == 4, 'label'] = '녹오염'
submit.loc[submit['label'] == 5, 'label'] = '들뜸'
submit.loc[submit['label'] == 6, 'label'] = '면불량'
submit.loc[submit['label'] == 7, 'label'] = '몰딩수정'
submit.loc[submit['label'] == 8, 'label'] = '반점'
submit.loc[submit['label'] == 9, 'label'] = '석고수정'
submit.loc[submit['label'] == 10, 'label'] = '오염'
submit.loc[submit['label'] == 11, 'label'] = '오타공'
submit.loc[submit['label'] == 12, 'label'] = '울음'
submit.loc[submit['label'] == 13, 'label'] = '이음부불량'
submit.loc[submit['label'] == 14, 'label'] = '창틀,문틀수정'
submit.loc[submit['label'] == 15, 'label'] = '터짐'
submit.loc[submit['label'] == 16, 'label'] = '틈새과다'
submit.loc[submit['label'] == 17, 'label'] = '피스'
submit.loc[submit['label'] == 18, 'label'] = '훼손'

In [13]:
submit.head()

,id,label
0,TEST_000,훼손
1,TEST_001,오염
2,TEST_002,훼손
3,TEST_003,훼손
4,TEST_004,오염


In [14]:
submit.to_csv('./results/'+str(np.mean(k_val_loss_plot)) + '.csv', index=False)